# Example receiving .csv files and doing aggregation on sales by state
<font color='steelblue'>
<br>
<span style="font-family:Arial; font-size:1.6em;">
Streaming .csv files and perform aggregation on sales data<br>
<br>
Steps to test the script: <br>
<ol>
    <li> Run all cells in this notebook</li>
    <li> Once the script reaches awaitTermination - keep eye on the "Jupyter Notebook console"</li>
    <li> Copy file datasets/sales1.csv to Labs/droplocation</li>
    <li> Observe the aggreation report in the console (takes 1 minute)</li>
    <li> Copy file datasets/sales2.csv to Labs/droplocation</li>
    <li> Observe the aggreation report in the console</li>
</ol>
 </span>

</font>
<br>
<font color='gray'>
<span style="font-family:Arial; font-size:1.4em;">
Following are the steps in the processing:<br>
    <ul>
        <li><strong>Schema:</strong> Schema definition of the csv file</li>
        <li><strong>Read Stream:</strong> Define read stream and set it to dataframe</li>
        <li><strong>Aggregate:</strong> Aggregate data based on sales and assign to aggregate dataframe</li>
        <li><strong>Write Stream:</strong> Write the aggregate dataframe as a stream to console</li>
    </ul>
</span>
</font>

In [ ]:
# Set up the environment for using pyspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import *

In [ ]:
# Want to write the print statements to console rather
# than output of jupyter cell - redirect stdout to connsole
import sys
sys.stdout = open(1, 'w')

In [ ]:
spark = SparkSession\
        .builder\
        .appName("Streaming Output modes")\
        .getOrCreate()

In [ ]:
 spark.sparkContext.setLogLevel("ERROR")

## Define schema for the data in .csv file

In [ ]:
# InferSchema not yet available in spark structured streaming
# (it is available in static dataframes)
# We explicity state the schema of the input data

schema = StructType([StructField("product", StringType(), True),
                     StructField("city", StringType(), True),
                     StructField("state", StringType(), True),
                     StructField("country", StringType(), True),
                     StructField("sales", StringType(), True)
                    ])

## Define the read stream
<h3>
<ol>
    <li>Read the stream into dataframe</li>
    <li>Since the csv data includes a header row, we specify that here</li>
    <li>We state the schema to use and the location of the csv files</li>
</ol>
</h3>

In [ ]:
fileStreamDF = spark.readStream\
                           .option("header", "false")\
                           .schema(schema)\
                           .csv("../droplocation")

In [ ]:
# Check whether input data is streaming or not
print(" ")
print("Is the stream ready?")
print(fileStreamDF.isStreaming)

In [ ]:
# Print Schema
print(" ")
print("Schema of the input stream: ")
print(fileStreamDF.printSchema)

## Aggregate sales by state and put in new column "tot_sales"

<font color='teal'>
    <span style="font-family:Arial; font-size:1.6em;">
        <i>If the output mode is <b>update</b> then the above code will give  error (sort is supported only on complete mode)<br> use the following code with update mode (comment the above code)</i>
    </span>
</font>

In [ ]:
salesDF = fileStreamDF.groupBy("state")\
                      .agg({"sales": "sum"})\
                      .withColumnRenamed("sum(sales)", "tot_sales")

## Write the aggregate dataframe to console

In [ ]:
query = salesDF.writeStream\
                  .outputMode('complete')\
                  .format('console')\
                  .option("truncate", "false")\
                  .option("numRows", 55)\
                  .start()

In [ ]:
query.awaitTermination()